## 4. `CombinedMemory`
- **Purpose**: Combines multiple memory types (e.g., buffer + summary, static + conversational).
- **Use‑case**: Agents needing both immediate context and long‑term knowledge.
- **Main difference**: Offers composite memory strategy; e.g.,
  static data via `SimpleMemory` + chat history via `ConversationBufferMemory` or `ConversationSummaryMemory`.

In [1]:
# ================================
# Step 1: Install dependencies
# ================================
!pip install -q langchain langchain-groq langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.6 MB/s eta 0:00:00


In [2]:
# ================================
# Step 2: Import libraries
# ================================
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.memory import (
    ConversationSummaryMemory,
    SimpleMemory,
    CombinedMemory
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from IPython.display import Markdown, display
from google.colab import userdata

In [4]:
# ================================
# Step 3: Setup LLMs (Groq + OpenAI)
# ================================
groq_key = userdata.get("GROQ_API_KEY")
openai_key = userdata.get("OPENAI_API_KEY3")

# Groq for response generation
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=groq_key,
    temperature=0.3,
)

# OpenAI (or lightweight HuggingFace model) for summarization
summarizer_llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=openai_key,
    temperature=0.3,
)

In [5]:
# ================================
# Step 4: Setup CombinedMemory
# ================================

# Static memory — constant data like user profile
profile_memory = SimpleMemory(memories={
    "user_name": "Dhruv",
    "user_role": "Machine Learning Engineer"
})

# Dynamic summary memory — for chat history
summary_memory = ConversationSummaryMemory(
    llm=summarizer_llm,
    return_messages=True
)

# Combine both memories
combined_memory = CombinedMemory(
    memories=[profile_memory, summary_memory]
)

<ipython-input-5-1506891984>:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(
/usr/local/lib/python3.11/dist-packages/langchain/memory/combined.py:40: UserWarning: When using CombinedMemory, input keys should be so the input is known.  Was not set on llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7c24703ffbd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7c24703fd4d0>, root_client=<openai.OpenAI object at 0x7c24703ff610>, root_async_client=<openai.AsyncOpenAI object at 0x7c24700d4c10>, model_name='gpt-4o-mini', temperature=0.3, model_kwargs={}, openai_api_key=SecretStr('**********')) chat_memory=InMemoryChatMessageHistory(messages=[]) return_messages=True
  warnings.warn(


In [6]:
# ================================
# Step 5: Define Prompt Template
# ================================
system_msg = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. The user's name is {user_name} and they are a {user_role}. "
    "Always personalize your replies. Use prior conversation (if any) to respond intelligently."
)

human_msg = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([
    system_msg,
    MessagesPlaceholder(variable_name="history"),  # injects summarized history
    human_msg
])

In [7]:
# ================================
# Step 6: Chat Loop (Manual)
# ================================
user_inputs = [
    "Hey, can you explain how a neural network learns?",
    "Now tell me how backpropagation works.",
    "Do you think it’s better than decision trees?"
]

for input_text in user_inputs:
    # Step 1: Load memory values
    memory_vars = combined_memory.load_memory_variables({})

    # Step 2: Format messages using both memory parts
    messages = chat_prompt.format_messages(
        input=input_text,
        **memory_vars
    )

    # Step 3: Invoke Groq for final response
    response = llm.invoke(messages)

    # Step 4: Save to summary memory only (SimpleMemory doesn't change)
    summary_memory.save_context(
        {"input": input_text},
        {"output": response.content}
    )

    # Step 5: Display result
    display(Markdown(f"### ❓ User: {input_text}"))
    display(Markdown(f"**🤖 Assistant:** {response.content}"))

### ❓ User: Hey, can you explain how a neural network learns?

**🤖 Assistant:** Hi Dhruv, nice to chat with you about neural networks. As a Machine Learning Engineer, you're likely familiar with the basics, but I'll provide a detailed overview of how a neural network learns.

A neural network learns through a process called backpropagation, which involves the following steps:

1. **Forward Pass**: The network receives input data, which flows through the layers, performing calculations and transformations. The output is calculated based on the current weights and biases of the network.
2. **Error Calculation**: The difference between the predicted output and the actual output (target) is calculated using a loss function, such as mean squared error or cross-entropy.
3. **Backward Pass**: The error is propagated backwards through the network, adjusting the weights and biases at each layer to minimize the loss. This is done using the chain rule of calculus.
4. **Weight Update**: The weights and biases are updated based on the calculated error and the learning rate, which controls how quickly the network learns.
5. **Optimization**: The network repeats the forward and backward passes, adjusting the weights and biases to minimize the loss. This process is typically performed using an optimization algorithm, such as stochastic gradient descent (SGD), Adam, or RMSProp.

The key components that enable a neural network to learn are:

* **Activation functions**: Introduce non-linearity to the network, allowing it to learn complex relationships between inputs and outputs.
* **Loss functions**: Measure the difference between predicted and actual outputs, guiding the network to minimize the error.
* **Optimization algorithms**: Adjust the weights and biases to minimize the loss, using techniques such as gradient descent and momentum.

As a Machine Learning Engineer, you're likely working with popular deep learning frameworks like TensorFlow or PyTorch, which provide efficient implementations of these concepts. Do you have any specific questions about implementing neural networks or optimizing their performance, Dhruv?

### ❓ User: Now tell me how backpropagation works.

**🤖 Assistant:** Dhruv, as a Machine Learning Engineer, you're likely familiar with the basics of neural networks, so I'll dive straight into the details of backpropagation. Backpropagation is an essential algorithm in training neural networks, and it's used to minimize the error between the network's predictions and the actual outputs.

The backpropagation process involves the following steps:

1. **Forward Pass**: First, the network processes the input data and produces an output. This output is then compared to the actual output to calculate the error.
2. **Error Calculation**: The error is calculated using a loss function, such as mean squared error or cross-entropy. This error represents the difference between the predicted output and the actual output.
3. **Backward Pass**: The error is then propagated backwards through the network, adjusting the weights and biases of each layer to minimize the error. This is done using the chain rule of calculus.
4. **Weight Update**: During the backward pass, the gradients of the loss function with respect to each weight and bias are calculated. These gradients are then used to update the weights and biases to minimize the error.

The key components that enable backpropagation are:

* **Activation Functions**: These introduce non-linearity into the network, allowing it to learn complex relationships between inputs and outputs.
* **Loss Functions**: These measure the difference between the predicted output and the actual output, providing a way to evaluate the network's performance.
* **Optimization Algorithms**: These, such as stochastic gradient descent (SGD), Adam, or RMSProp, are used to update the weights and biases based on the calculated gradients.

In backpropagation, the gradients are calculated using the following formula:

δ = (dE/dy) \* (dy/dz) \* (dz/dw)

where δ is the gradient of the loss function with respect to the weight, dE/dy is the derivative of the loss function with respect to the output, dy/dz is the derivative of the output with respect to the weighted sum, and dz/dw is the derivative of the weighted sum with respect to the weight.

As someone who works with frameworks like TensorFlow or PyTorch, you're likely familiar with how these libraries implement backpropagation and provide tools for optimizing neural networks. If you have any specific questions about implementing or optimizing backpropagation in your projects, feel free to ask, Dhruv!

### ❓ User: Do you think it’s better than decision trees?

**🤖 Assistant:** Dhruv, as a Machine Learning Engineer, you're likely familiar with the strengths and weaknesses of various algorithms. Backpropagation is a key component of neural networks, which can be more powerful than decision trees in certain scenarios.

Neural networks, trained using backpropagation, can learn complex, non-linear relationships between inputs and outputs, making them particularly well-suited for tasks like image classification, natural language processing, and speech recognition. They can also handle large datasets and high-dimensional feature spaces.

In contrast, decision trees are often more interpretable and can be more efficient to train, but they can struggle with complex relationships and may not generalize as well to new, unseen data. Additionally, decision trees can be prone to overfitting, especially when dealing with noisy or high-dimensional data.

That being said, decision trees can be a great choice for certain problems, such as feature selection, data exploration, or when interpretability is crucial. They're also often used as a component of more complex models, like random forests or gradient boosting machines.

So, it ultimately depends on the specific problem you're trying to solve, Dhruv. If you're working on a task that requires learning complex patterns in data, neural networks trained with backpropagation might be a better choice. But if you need a more interpretable model or are working with a smaller dataset, decision trees could be a better fit.

Do you have a specific project in mind where you're considering using backpropagation or decision trees? I'd be happy to help you weigh the pros and cons.